# 01_環境設定

## 🎯 學習目標
- 完成 Python 開發環境配置
- 安裝必要的 LangChain 套件
- 設定 API 金鑰並測試連線
- 驗證環境是否正常運作

---

## 1. Python 環境配置

### 檢查 Python 版本
建議使用 Python 3.9 或以上版本

In [ ]:
import sys
print(f"Python 版本: {sys.version}")

# 檢查版本是否符合要求
if sys.version_info >= (3, 9):
    print("✅ Python 版本符合要求")
else:
    print("❌ Python 版本過低，建議升級到 3.9+")

### 虛擬環境設定 (選擇性)

如果您希望使用虛擬環境，請在終端機執行以下指令：

```bash
# 建立虛擬環境
python -m venv storm_writing_env

# 啟動虛擬環境 (Windows)
storm_writing_env\Scripts\activate

# 啟動虛擬環境 (macOS/Linux)
source storm_writing_env/bin/activate

# 安裝 Jupyter
pip install jupyter

# 啟動 Jupyter Notebook
jupyter notebook
```

---

## 2. 套件安裝

### 核心套件清單

In [ ]:
# 必要套件安裝
!pip install langchain langchain-openai langchain-community python-dotenv requests beautifulsoup4 duckduckgo-search

In [ ]:
# 驗證套件安裝
import importlib

packages = {
    'langchain': 'langchain',
    'langchain_openai': 'langchain-openai',
    'langchain_community': 'langchain-community',
    'dotenv': 'python-dotenv',
    'requests': 'requests',
    'bs4': 'beautifulsoup4',
    'duckduckgo_search': 'duckduckgo-search'
}

print("📦 套件安裝檢查:")
for package, install_name in packages.items():
    try:
        module = importlib.import_module(package)
        if hasattr(module, '__version__'):
            version = module.__version__
        else:
            version = "已安裝"
        print(f"✅ {install_name}: {version}")
    except ImportError:
        print(f"❌ {install_name}: 未安裝")

---

## 3. API 金鑰設定

### 建立 .env 檔案
在專案根目錄建立 `.env` 檔案，內容如下：

In [ ]:
# 建立範例 .env 檔案
import os

env_content = '''
# OpenAI API 設定
OPENAI_API_KEY=your-openai-api-key-here

# 可選的其他 API
ANTHROPIC_API_KEY=your-anthropic-api-key-here
SERPAPI_API_KEY=your-serpapi-key-here
'''

# 檢查是否已存在 .env 檔案
if not os.path.exists('.env'):
    with open('.env', 'w', encoding='utf-8') as f:
        f.write(env_content)
    print("✅ 已建立 .env 範例檔案")
    print("⚠️  請編輯 .env 檔案，填入您的實際 API 金鑰")
else:
    print("ℹ️  .env 檔案已存在")

### 載入環境變數

In [ ]:
from dotenv import load_dotenv
import os

# 載入環境變數
load_dotenv()

# 檢查 API 金鑰是否已設定
openai_key = os.getenv('OPENAI_API_KEY')
if openai_key and openai_key != 'your-openai-api-key-here':
    print("✅ OpenAI API 金鑰已設定")
    print(f"金鑰前綴: {openai_key[:10]}...")
else:
    print("❌ 請在 .env 檔案中設定您的 OpenAI API 金鑰")
    print("💡 獲取 API 金鑰: https://platform.openai.com/api-keys")

---

## 4. 基礎功能測試

### OpenAI API 連線測試

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage

# 初始化 OpenAI 客戶端
try:
    llm = ChatOpenAI(
        model="gpt-3.5-turbo",
        temperature=0.7,
        api_key=os.getenv('OPENAI_API_KEY')
    )
    
    # 測試基本功能
    test_message = HumanMessage(content="請用一句話說明什麼是人工智慧")
    response = llm.invoke([test_message])
    
    print("✅ OpenAI API 連線成功")
    print(f"回應: {response.content}")
    
except Exception as e:
    print(f"❌ OpenAI API 連線失敗: {str(e)}")
    print("💡 請檢查 API 金鑰是否正確設定")

### 網路搜尋功能測試

In [ ]:
from duckduckgo_search import DDGS
import requests

# 測試 DuckDuckGo 搜尋
try:
    ddgs = DDGS()
    results = list(ddgs.text("人工智慧最新發展", max_results=3))
    
    print("✅ 網路搜尋功能正常")
    print(f"搜尋到 {len(results)} 筆結果")
    
    for i, result in enumerate(results, 1):
        print(f"{i}. {result['title'][:50]}...")
        
except Exception as e:
    print(f"❌ 網路搜尋功能異常: {str(e)}")

### 網頁內容擷取測試

In [ ]:
import requests
from bs4 import BeautifulSoup

def extract_text_from_url(url, max_length=500):
    """從網址擷取文字內容"""
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
        }
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # 移除 script 和 style 元素
        for script in soup(["script", "style"]):
            script.decompose()
        
        # 擷取文字內容
        text = soup.get_text()
        lines = (line.strip() for line in text.splitlines())
        chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
        text = ' '.join(chunk for chunk in chunks if chunk)
        
        return text[:max_length] if len(text) > max_length else text
        
    except Exception as e:
        return f"擷取失敗: {str(e)}"

# 測試網頁內容擷取
test_url = "https://zh.wikipedia.org/wiki/%E4%BA%BA%E5%B7%A5%E6%99%BA%E8%83%BD"
content = extract_text_from_url(test_url)

if "擷取失敗" not in content:
    print("✅ 網頁內容擷取功能正常")
    print(f"擷取內容預覽: {content[:100]}...")
else:
    print(f"❌ {content}")

---

## 5. 工具函數準備

### 建立基礎工具類別

In [ ]:
class STORMTools:
    """STORM 系統基礎工具集"""
    
    def __init__(self, openai_api_key=None):
        self.openai_api_key = openai_api_key or os.getenv('OPENAI_API_KEY')
        self.llm = ChatOpenAI(
            model="gpt-3.5-turbo",
            temperature=0.7,
            api_key=self.openai_api_key
        )
        
    def search_web(self, query, max_results=5):
        """網路搜尋功能"""
        try:
            ddgs = DDGS()
            results = list(ddgs.text(query, max_results=max_results))
            return results
        except Exception as e:
            print(f"搜尋錯誤: {str(e)}")
            return []
    
    def extract_content(self, url, max_length=1000):
        """擷取網頁內容"""
        return extract_text_from_url(url, max_length)
    
    def generate_text(self, prompt):
        """使用 LLM 生成文字"""
        try:
            message = HumanMessage(content=prompt)
            response = self.llm.invoke([message])
            return response.content
        except Exception as e:
            return f"生成錯誤: {str(e)}"
    
    def test_all_functions(self):
        """測試所有功能"""
        print("🧪 開始功能測試...")
        
        # 測試搜尋
        search_results = self.search_web("人工智慧", 2)
        print(f"✅ 搜尋功能: 找到 {len(search_results)} 筆結果")
        
        # 測試內容擷取
        if search_results:
            content = self.extract_content(search_results[0]['href'])
            print(f"✅ 內容擷取: 擷取了 {len(content)} 個字元")
        
        # 測試文字生成
        text = self.generate_text("請簡述人工智慧的定義")
        print(f"✅ 文字生成: {text[:50]}...")
        
        print("🎉 所有功能測試完成！")

# 建立工具實例
tools = STORMTools()
print("✅ STORM 工具類別已建立")

### 完整功能測試

In [ ]:
# 執行完整測試
tools.test_all_functions()

---

## 6. 環境檢查總結

### 檢查清單

In [ ]:
import sys

def environment_check():
    """完整環境檢查"""
    checks = []
    
    # Python 版本檢查
    python_ok = sys.version_info >= (3, 9)
    checks.append(("Python 3.9+", python_ok))
    
    # 套件檢查
    packages = ['langchain', 'langchain_openai', 'dotenv', 'requests', 'bs4', 'duckduckgo_search']
    for package in packages:
        try:
            importlib.import_module(package)
            checks.append((f"{package} 套件", True))
        except ImportError:
            checks.append((f"{package} 套件", False))
    
    # API 金鑰檢查
    api_key = os.getenv('OPENAI_API_KEY')
    api_ok = api_key and api_key != 'your-openai-api-key-here'
    checks.append(("OpenAI API 金鑰", api_ok))
    
    # 顯示結果
    print("📋 環境檢查結果:")
    print("=" * 30)
    
    all_passed = True
    for item, status in checks:
        icon = "✅" if status else "❌"
        print(f"{icon} {item}")
        if not status:
            all_passed = False
    
    print("=" * 30)
    if all_passed:
        print("🎉 環境設定完成！可以開始下一步開發")
    else:
        print("⚠️  請解決上述問題後再繼續")
    
    return all_passed

# 執行檢查
environment_check()

---

## 🎯 本章小結

### 完成項目
- ✅ Python 環境確認
- ✅ 必要套件安裝
- ✅ API 金鑰設定
- ✅ 基礎功能測試
- ✅ 工具類別建立

### 下一步預告
在下一個 notebook「02_手動版_研究智能體」中，我們將：
1. 實作 STORM 的第一個階段：知識探索
2. 開發研究智能體的核心功能
3. 實現多角度問題生成
4. 建立資料收集與驗證機制

### 故障排除
如果遇到問題，請檢查：
1. **網路連線**: 確保可以正常訪問網際網路
2. **API 配額**: 確認 OpenAI 帳戶有足夠的使用配額
3. **防火牆設定**: 某些企業網路可能會阻擋 API 請求
4. **套件版本**: 如果有相容性問題，可以嘗試指定特定版本

---

*環境設定完成！現在您已經具備開發 STORM 系統所需的所有工具。讓我們開始實際的智能體開發之旅！*